# *datalab* Python API example notebook

This notebook will run through some of the common operations you might want to perform with the Python API.

It uses the public deployment at https://demo.datalab-org.io via its API endpoint at https://demo-api.datalab-org.io. You can register an account there with a valid ORCID and generate an API key for yourself at https://demo-api.datalab-org.io/generate-api-key, after logging in.

In this notebook, we'll add some fake data to the public deployment, upload some files and create some data blocks.

## Getting started

First, we need to tell the client which *datalab* instance to connect to. You need to use the API URL, rather than the URL for the web UI for this, though the code can figure this out depending on your *datalab* instance.

The client will need an API key to be set with the `DATALAB_API_KEY` environment variable. Ideally you should set this outside your notebook so you do not have to save it in plain text anywhere, but for now we will just set it in Python (don't worry, this key is not active).

In [10]:
import os

os.environ["DATALAB_API_KEY"] = "FRWwHSTLocXnihBTZOoGbnRUqyHZEtFt"

We can also load the Bokeh Jupyter plugin at this point, so that blocks will appear inline:

In [2]:
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

Now we can import the client and begin to use it:

In [3]:
from datalab_api import DatalabClient

client = DatalabClient("demo-api.datalab-org.io")

We can print some information about this instance:

In [4]:
client.get_info(display=True);

{
│   'data': {
│   │   'attributes': {
│   │   │   'api_version': '0.1.0',
│   │   │   'available_api_versions': [...],
│   │   │   'datamodel_version': '0.3.2',
│   │   │   'homepage': None,
│   │   │   'identifier_prefix': 'pub',
│   │   │   'issue_tracker': None,
│   │   │   'maintainer': None,
│   │   │   'query': '',
│   │   │   'server_version': '0.3.2',
│   │   │   'source_repository': None,
│   │   │   'timestamp': '2024-05-28T16:21:48.618972'
│   │   },
│   │   'id': '/',
│   │   'type': 'info'
│   },
│   'links': {'self': 'https://demo-api.datalab-org.io/info'},
│   'meta': {
│   │   'api_version': '0.1.0',
│   │   'available_api_versions': ['0.1.0'],
│   │   'datamodel_version': '0.3.2',
│   │   'query': '',
│   │   'server_version': '0.3.2',
│   │   'timestamp': '2024-05-28T16:21:48.619065'
│   }
}

Or about our authentication status:

In [5]:
client.authenticate(display=True);

{
│   'account_status': 'unverified',
│   'contact_email': None,
│   'display_name': 'Matthew L Evans',
│   'identities': [
│   │   {
│   │   │   'display_name': 'Matthew L Evans',
│   │   │   'identifier': '0000-0002-1182-9098',
│   │   │   'identity_type': 'orcid',
│   │   │   'name': '0000-0002-1182-9098',
│   │   │   'verified': True
│   │   }
│   ],
│   'immutable_id': '65e7502b42e15342c7ff34ef',
│   'last_modified': None,
│   'managers': None,
│   'relationships': None,
│   'role': 'admin',
│   'type': 'people'
}

Or get information about which blocks are implemented:

In [11]:
client.get_block_info(display=True);

[
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'Add a rich text comment to the document.',
│   │   │   'name': 'Comment',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'comment',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'Display an image or a video of a supported format.',
│   │   │   'name': 'Media',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'media',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'Visualize XRD patterns and perform simple baseline corrections.',
│   │   │   'name': 'Powder XRD',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'xrd',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': "This block can plot data from electrochemical cycling experiments from many different cycler's file "+8,
│   │   │   'name': 'Electrochemical cycling',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'cycle',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'Visualize 1D Raman spectroscopy data.',
│   │   │   'name': 'Raman spectroscopy',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'raman',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'A simple NMR block for visualizing 1D NMR data from Bruker projects.',
│   │   │   'name': 'NMR',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'nmr',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'A placeholder block type when the requested block is not supported by the current version of the ser'+4,
│   │   │   'name': 'Not Supported',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'notsupported',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'Read and visualize mass spectrometry data as a grid plot per channel',
│   │   │   'name': 'Mass spectrometry',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'ms',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': None,
│   │   │   'description': 'Virtual LLM assistant block allows you to converse with your data.',
│   │   │   'name': 'Whinchat assistant',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'chat',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'This block can plot electrochemical impedance spectroscopy (EIS) data from Ivium .txt files',
│   │   │   'name': 'EIS',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'eis',
│   │   'type': 'block_type'
│   },
│   {
│   │   'attributes': {
│   │   │   'accepted_file_extensions': [...],
│   │   │   'description': 'This block will load tabular data from common plain text files and allow you to create simple scatte'+30,
│   │   │   'name': 'Tabular Data Block',
│   │   │   'version': '0.1.0'
│   │   },
│   │   'id': 'tabular',
│   │   'type': 'block_type'
│   }
]

## Getting entries

To search for items with a query string:

In [6]:
items = client.search_items("test", display=True);

┏━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ type ┃ ID             ┃ refcode    ┃ name                           ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  S   │ test           │ pub:HOPZJD │ test                           │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ my_test_sample │ pub:IGCYOI │ Test Sample                    │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ test-claude    │ pub:JMGOTA │                                │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ test_sample    │ pub:JTNEWW │ G1500                          │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ JPN01          │ pub:AIMCHP │ Siloxane Reaction              │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ jdb102         │ pub:WRIZSB │ Sodium cobalt oxide            │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  C   │ jdb2_e1_c1     │ pub:TBBADR │ NaCoO2 coin cell               │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ jdb1           │ pub:VRSMCI │ sodium cobalt oxide made by s… │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  S   │ jdb2           │ pub:HPPPKI │ sodium cobalt oxide made by s… │
├──────┼────────────────┼────────────┼────────────────────────────────┤
│  C   │ NMC811_e1_c1   │ pub:EAJBJB │ NMC-811 coin cell              │
└──────┴────────────────┴────────────┴────────────────────────────────┘

To list all items of a specified type:

In [7]:
items = client.get_items(item_type="samples", page_limit=50, display=True);

┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ type ┃ ID                                  ┃ refcode    ┃ name                           ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  S   │ test_salt                           │ pub:BACDYQ │ Salt for my dinner             │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ test_new_bc1                        │ pub:KTXPAW │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ test_new_bc                         │ pub:BCVWFA │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ test_new                            │ pub:MYHNKR │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ jdb10                               │ pub:LKKIBV │ sodium cobalt oxide            │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ AABBCC                              │ pub:RGUNUV │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ PCFETT                              │ pub:PCFETT │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ DPCFKY                              │ pub:DPCFKY │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ LJCQZY                              │ pub:LJCQZY │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ WGZKOB                              │ pub:WGZKOB │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ AVWKIT                              │ pub:AVWKIT │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ AFEVNW                              │ pub:AFEVNW │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ NYAWFN                              │ pub:NYAWFN │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ AVKTET                              │ pub:AVKTET │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ XTTENL                              │ pub:XTTENL │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ HUQXTC                              │ pub:HUQXTC │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ WUKFBU                              │ pub:WUKFBU │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ MYPSNX                              │ pub:MYPSNX │                                │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ xu_et_al_2022_solid_state_synthesis │ pub:DUSORL │ Xu et al (2022) - Solid state… │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│  S   │ my_test_sample                      │ pub:IGCYOI │ Test Sample                    │
├──────┼─────────────────────────────────────┼────────────┼────────────────────────────────┤
│

To retrieve detailed information on a specific item with id = 'test'

In [8]:
item = client.get_item(item_id="lab_notebook_sample_02", display=True)

{
│   'blocks_obj': {},
│   'chemform': 'Na3Ni2MnO6',
│   'collections': [],
│   'creator_ids': ['65e7502b42e15342c7ff34ef'],
│   'creators': [{'display_name': 'Matthew L Evans'}],
│   'date': '2024-05-09T20:17:14.920000',
│   'description': 'Handwritten notes on solid state synthesis procedure',
│   'display_order': [],
│   'file_ObjectIds': [],
│   'files': [],
│   'immutable_id': '663d2f4a11db676610ae7715',
│   'item_id': 'lab_notebook_sample_02',
│   'last_modified': None,
│   'name': 'Lab notebook - Xu et al solid state synthesis',
│   'refcode': 'pub:GIBNNA',
│   'relationships': [],
│   'revision': 1,
│   'revisions': None,
│   'synthesis_constituents': [
│   │   {'item': {...}, 'quantity': 0.3397, 'unit': 'g'},
│   │   {'item': {...}, 'quantity': 0.2382, 'unit': 'g'},
│   │   {'item': {...}, 'quantity': 0.5107, 'unit': 'g'}
│   ],
│   'synthesis_description': 'Solid state synthesis of Na3Ni2MnO6 from Na2CO3, NiO, Mn2O3 and O2',
│   'type': 'samples'
}

To retrieve detailed information on a specific item, and load and show its data blocks:

In [ ]:
client.get_item(item_id="NMC811_e1", display=True, load_blocks=True);

In [11]:
client.get_item(item_id="NMC811_e1_c1", load_blocks=True, display=True);

{
│   'active_ion_charge': 1.0,
│   'blocks_obj': {
│   │   'e6te202qpfc7rei': {
│   │   │   'block_id': 'e6te202qpfc7rei',
│   │   │   'blocktype': 'cycle',
│   │   │   'bokeh_plot_data': {...},
│   │   │   'collection_id': None,
│   │   │   'derivative_mode': None,
│   │   │   'file_id': '65d6f099fa74b62642328512',
│   │   │   'freeform_comment': '<p>Cell charged for 4 hours (to about 4.2 V) in order to partially delithiate.</p>',
│   │   │   'item_id': 'NMC811_e1_c1',
│   │   │   'p_spline': 5,
│   │   │   's_spline': 5,
│   │   │   'title': 'Electrochemical cycling',
│   │   │   'win_size_1': 1001,
│   │   │   'win_size_2': 101
│   │   }
│   },
│   'cell_format': 'coin',
│   'cell_format_description': 'CR2032',
│   'cell_preparation_description': '<p>Coin cell assembled in argon glovbox. GF/B glass fiber separator. Two 0.5mm stainless steel space'+38,
│   'characteristic_chemical_formula': 'LiNi0.8Mn0.1Co0.1O2',
│   'characteristic_mass': 12.08,
│   'characteristic_molar_mass': 97.2816449,
│   'collections': [{'collection_id': 'NMC-811_coin_cells', 'immutable_id': '65d6ed16fa74b626423284c5'}],
│   'creator_ids': ['65d6e50050726b088d328499', '000000000000000000000000'],
│   'creators': [{'contact_email': 'jdbocarsly@gmail.com', 'display_name': 'jdbocarsly@gmail.com'}],
│   'date': '2024-02-22T06:38:00',
│   'description': '<p><strong>NOTE: this is a read-only example entry. </strong></p>\n<p>Coin cell used to test electroc'+56,
│   'display_order': ['e6te202qpfc7rei'],
│   'electrolyte': [{'item': {...}, 'quantity': 100.0, 'unit': 'µL'}],
│   'file_ObjectIds': ['65d6eea891cdacd6563284ff'],
│   'files': [
│   │   {
│   │   │   'blocks': [...],
│   │   │   'creator_ids': [...],
│   │   │   'creators': None,
│   │   │   'extension': '.mpr',
│   │   │   'immutable_id': '65d6eea891cdacd6563284ff',
│   │   │   'is_live': False,
│   │   │   'item_ids': [...],
│   │   │   'last_modified': '2024-02-22T06:50:16.177000',
│   │   │   'last_modified_remote': None,
│   │   │   'location': '/app/files/65d6eea891cdacd6563284ff/NMC811_1mAh_cm-2_5cycles4.2V-5cycles4.5V_Con20.mpr',
│   │   │   'metadata': {...},
│   │   │   'name': 'NMC811_1mAh_cm-2_5cycles4.2V-5cycles4.5V_Con20.mpr',
│   │   │   'original_name': 'NMC811_1mAh_cm-2_5cycles4.2V-5cycles4.5V_Con20.mpr',
│   │   │   'relationships': None,
│   │   │   'representation': None,
│   │   │   'revision': 1,
│   │   │   'revisions': None,
│   │   │   'size': 3138708,
│   │   │   'source': 'uploaded',
│   │   │   'source_path': None,
│   │   │   'source_server_name': None,
│   │   │   'time_added': '2024-02-22T06:50:16.177000',
│   │   │   'type': 'files',
│   │   │   'url_path': None
│   │   }
│   ],
│   'immutable_id': '65d6ec2791cdacd6563284c8',
│   'item_id': 'NMC811_e1_c1',
│   'last_modified': '2024-02-22T07:47:32.186000',
│   'name': 'NMC-811 coin cell',
│   'negative_electrode': [{'item': {...}, 'quantity': 1.0, 'unit': 'disc'}],
│   'positive_electrode': [{'item': {...}, 'quantity': 206.0, 'unit': 'mg'}],
│   'refcode': 'pub:EAJBJB',
│   'relationships': [
│   │   {
│   │   │   'description': 'Is a constituent of',
│   │   │   'immutable_id': None,
│   │   │   'item_id': 'NMC811_e1',
│   │   │   'refcode': None,
│   │   │   'relation': 'is_part_of',
│   │   │   'type': 'samples'
│   │   },
│   │   {
│   │   │   'description': 'Is a constituent of',
│   │   │   'immutable_id': None,
│   │   │   'item_id': 'AB000514',
│   │   │   'refcode': None,
│   │   │   'relation': 'is_part_of',
│   │   │   'type': 'starting_materials'
│   │   },
│   │   {
│   │   │   'description': 'Is a constituent of',
│   │   │   'immutable_id': None,
│   │   │   'item_id': 'AN000340',
│   │   │   'refcode': None,
│   │   │   'relation': 'is_part_of',
│   │   │   'type': 'starting_materials'
│   │   },
│   │   {
│   │   │   'description': 'Is a member of',
│   │   │   'immutable_id': '65d6ed16fa74b626423284c5',
│   │   │   'item_id': None,
│   │   │   'refcode': None,
│   │   │   'relation': None,
│   │   │

## Creating entries

To create a new item with data that matches the specified item_type schema

In [13]:
json_data = {"chemform": "NaCl", "name": "Salt for my dinner"}
client.create_item(item_id="test_salt", item_type="samples", item_data=json_data)

{'collections': [],
 'creator_ids': [{'$oid': '65e7502b42e15342c7ff34ef'}],
 'creators': [{'contact_email': None, 'display_name': 'Matthew L Evans'}],
 'date': '2024-05-28T16:23:05.848109',
 'item_id': 'test_salt',
 'name': 'Salt for my dinner',
 'nblocks': 0,
 'refcode': 'pub:DXSRGU',
 'type': 'samples'}

To attach a file to an item and obtain the uploaded file ID:

In [14]:
from pathlib import Path

file_response = client.upload_file(
    item_id="test_salt",
    file_path=Path.cwd().parent.parent.parent
    / "datalab"
    / "pydatalab"
    / "example_data"
    / "echem"
    / "jdb11-1_c3_gcpl_5cycles_2V-3p8V_C-24_data_C09.mpr",
    display=True,
)
file_id = file_response["file_id"]

{
│   'file_id': '665604eb8abd3b76edae78df',
│   'file_information': {
│   │   '_id': {'$oid': '665604eb8abd3b76edae78df'},
│   │   'blocks': [],
│   │   'creator_ids': [{...}],
│   │   'creators': None,
│   │   'extension': '.mpr',
│   │   'immutable_id': {'$oid': '665604eb8abd3b76edae78df'},
│   │   'is_live': False,
│   │   'item_ids': ['test_salt'],
│   │   'last_modified': '2024-05-28T16:23:07.041000',
│   │   'last_modified_remote': None,
│   │   'location': '/app/files/665604eb8abd3b76edae78df/jdb11-1_c3_gcpl_5cycles_2V-3p8V_C-24_data_C09.mpr',
│   │   'metadata': {},
│   │   'name': 'jdb11-1_c3_gcpl_5cycles_2V-3p8V_C-24_data_C09.mpr',
│   │   'original_name': 'jdb11-1_c3_gcpl_5cycles_2V-3p8V_C-24_data_C09.mpr',
│   │   'relationships': None,
│   │   'representation': None,
│   │   'revision': 1,
│   │   'revisions': None,
│   │   'size': 2465718,
│   │   'source': 'uploaded',
│   │   'source_path': None,
│   │   'source_server_name': None,
│   │   'time_added': '2024-05-28T16:23:07.041000',
│   │   'type': 'files',
│   │   'url_path': None
│   },
│   'is_update': '',
│   'status': 'success'
}

To create a data block for a sample and display the plot

In [15]:
block = client.create_data_block(
    item_id="test_salt", block_type="cycle", file_ids=file_id, display=False
);